In [1]:
%cd /content/drive/My Drive/Colab Notebooks/playing with btc

/content/drive/My Drive/Colab Notebooks/playing with btc


In [2]:
!pip install backtrader
!pip install pyfolio
# For plotting capabilities
!pip install backtrader[plotting]

In [3]:
!wget https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files/libta-lib0_0.4.0-oneiric1_amd64.deb -qO libta.deb
!wget https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files/ta-lib0-dev_0.4.0-oneiric1_amd64.deb -qO ta.deb
!dpkg -i libta.deb ta.deb
!pip install ta-lib

(Reading database ... 144654 files and directories currently installed.)
Preparing to unpack libta.deb ...
Unpacking libta-lib0 (0.4.0-oneiric1) over (0.4.0-oneiric1) ...
Preparing to unpack ta.deb ...
Unpacking ta-lib0-dev (0.4.0-oneiric1) over (0.4.0-oneiric1) ...
Setting up libta-lib0 (0.4.0-oneiric1) ...
Setting up ta-lib0-dev (0.4.0-oneiric1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Processing triggers for libc-bin (2.27-3ubuntu1.2) ...
/sbin/ldconfig.real: /usr/local/lib/python3.6/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link



In [4]:
%matplotlib inline
import pandas as pd
import backtrader as bt
import backtrader.indicators as btind
import backtrader.analyzers as btanalyzers
# import pyfolio as pf
# import talib
import datetime

from backtrader.feeds import PandasData



In [5]:
df = pd.read_csv('bitcoin/btc_1h_2017_2020.csv')
names = ['open', 'high', 'low', 'close', 'volume', 'open_time']
df = df[names]
df

,open,high,low,close,volume,open_time
0,4261.48,4313.62,4261.32,4308.83,47.181009,2017-08-17 04:00:00.000
1,4308.83,4328.69,4291.37,4315.32,23.234916,2017-08-17 05:00:00.000
2,4330.29,4345.45,4309.37,4324.35,7.229691,2017-08-17 06:00:00.000
3,4316.62,4349.99,4287.41,4349.99,4.443249,2017-08-17 07:00:00.000
4,4333.32,4377.85,4333.32,4360.69,0.972807,2017-08-17 08:00:00.000
...,...,...,...,...,...,...
27073,10454.88,10492.00,10447.53,10491.00,1672.365221,2020-09-22 17:00:00.000
27074,10490.99,10506.22,10484.34,10489.25,1796.497506,2020-09-22 18:00:00.000
27075,10489.25,10495.04,10467.36,10491.97,1198.053554,2020-09-22 19:00:00.000
27076,10491.98,10516.67,10480.00,10512.77,1272.055158,2020-09-22 20:00:00.000


# ticker and the start and end dates for testing

In [6]:
format = '%Y-%m-%d %H:%M:%S.%f'
df['open_time'] = pd.to_datetime(df['open_time'], format=format)
df.set_index('open_time', inplace=True)
df.head(2)

,open,high,low,close,volume
open_time,,,,,
2017-08-17 04:00:00,4261.48,4313.62,4261.32,4308.83,47.181009
2017-08-17 05:00:00,4308.83,4328.69,4291.37,4315.32,23.234916


In [7]:
# class to define the columns we will provide
class SignalData(PandasData):
    """
    Define pandas DataFrame structure
    """
    OHLCV = ['open', 'high', 'low', 'close', 'volume']
    cols = OHLCV

    # create lines
    lines = tuple(cols)

    # define parameters
    params = {c: -1 for c in cols}
    params.update({'datetime': None})
    params = tuple(params.items())

# Strategy

Use two moving averages and buy in the cross of the moving average, but the RSI must be above a certain range.

Then sell the position when the RSI is too high or when we achieve a certain value.

In [8]:
# define backtesting strategy class
class Strategy(bt.Strategy):
  params = dict()
  
  def __init__(self):

    self.btc = self.datas[0]
    self.close = self.btc.close
    self.ema_20 = btind.ExponentialMovingAverage(self.btc.close, period=20)
    self.ema_40 = btind.ExponentialMovingAverage(self.btc.close, period=40)
    # 1 if 20 crosses up 40 -1 if 20 crosses down 40
    self.cross = btind.CrossOver(self.ema_20, self.ema_40)

    self.order = None

  # logging function
  def log(self, txt):
    '''Logging function'''
    dt = self.datas[0].datetime.date(0).isoformat()
    print(f'{dt}, {txt}')

  def next(self):
    # not in the market
    if not self.position:  # not in the market
      if self.cross[0] > 0:
        self.order = self.buy()
        self.log(f'Buy at {self.btc.close[0]}')
        self.order = 'buying'

    elif self.order == 'buying': # position in market
      if self.cross[0] < 0:
        self.order = self.sell()
        self.log(f'Sell at {self.btc.close[0]}')




In [49]:

# instantiate SignalData class
data = SignalData(dataname=df['2020-05'])

In [50]:
# instantiate Cerebro, add strategy, data, initial cash, commission and pyfolio for performance analysis
cerebro = bt.Cerebro(stdstats = False)
cerebro.addstrategy(Strategy)
cerebro.adddata(data, name='BTC/USDT')
cerebro.broker.setcash(20000.0)
cerebro.broker.setcommission(commission=0.001)
cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')

# Analizers

In [51]:
# Analyzer
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='ta')
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe', riskfreerate=0.0, annualize=True, timeframe=bt.TimeFrame.Minutes)
cerebro.addanalyzer(bt.analyzers.VWR, _name='vwr')
cerebro.addanalyzer(bt.analyzers.SQN, _name='sqn')
cerebro.addanalyzer(bt.analyzers.Transactions, _name='txn')

In [52]:
# run the backtest
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
backtest_result = cerebro.run()
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 20000.00
2020-05-05, Buy at 8901.65
2020-05-09, Sell at 9655.28
2020-05-12, Buy at 8879.0
2020-05-15, Sell at 9316.42
2020-05-17, Buy at 9508.11
2020-05-19, Sell at 9537.78
2020-05-19, Buy at 9792.0
2020-05-20, Sell at 9497.99
2020-05-24, Buy at 9255.53
2020-05-24, Sell at 9159.9
2020-05-27, Buy at 9128.0
2020-05-31, Sell at 9448.27
Final Portfolio Value: 21047.28


In [53]:
strat = backtest_result[0]
from pprint import pprint

drawdown = strat.analyzers.drawdown.get_analysis()
pprint(drawdown)
sharpe_ratio = strat.analyzers.sharpe.get_analysis()
pprint(sharpe_ratio)
transactions = strat.analyzers.txn.get_analysis()
pprint(transactions)





AutoOrderedDict([('len', 422),
                 ('drawdown', 2.787582989021548),
                 ('moneydown', 603.5344100000038),
                 ('max',
                  AutoOrderedDict([('len', 422),
                                   ('drawdown', 4.536062278419471),
                                   ('moneydown', 982.0944100000052)]))])
OrderedDict([('sharperatio', 0.03343162622616251)])
OrderedDict([(datetime.datetime(2020, 5, 5, 1, 0),
              [[1, 8901.61, 0, 'BTC/USDT', -8901.61]]),
             (datetime.datetime(2020, 5, 9, 21, 0),
              [[-1, 9655.32, 0, 'BTC/USDT', 9655.32]]),
             (datetime.datetime(2020, 5, 12, 18, 0),
              [[1, 8879.0, 0, 'BTC/USDT', -8879.0]]),
             (datetime.datetime(2020, 5, 16, 0, 0),
              [[-1, 9315.96, 0, 'BTC/USDT', 9315.96]]),
             (datetime.datetime(2020, 5, 17, 5, 0),
              [[1, 9508.1, 0, 'BTC/USDT', -9508.1]]),
             (datetime.datetime(2020, 5, 19, 5, 0),
             

In [ ]:
%matplotlib inline
cerebro.plot(height=300, width=150)

In [15]:
cerebro.addobserver(bt.observers.BuySell)
cerebro.addobserver(bt.observers.Value)
cerebro.addobserver(bt.observers.DrawDown)
cerebro.addobserver(bt.observers.Trades)

cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='ta')
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe', riskfreerate=0.0, annualize=True, timeframe=bt.TimeFrame.Days)
cerebro.addanalyzer(bt.analyzers.VWR, _name='vwr')
cerebro.addanalyzer(bt.analyzers.SQN, _name='sqn')
cerebro.addanalyzer(bt.analyzers.Transactions, _name='txn')